In [1]:
import sqlite3
import pandas as pd
from pyhwpx import Hwp
import os
import re

#region 공통 데이터
#안전관리계획서1의 데이터 딕셔너리(엑셀의 열 / 데이터의 종류)
안전계획서1_dic ={
    "파일이름" : "",
    "현장명" : "",
    "현장소재지" : "",
    "공사금액" : 0.0,
    "보고서_날짜_년" : "",
    "보고서_날짜_월" : "",
    "시공자" : "",
    "설계자" : "",
    "공사개요_대상공사" : "",
    "공사개요_구조" : "",
    "공사개요_개소" : 0,
    "공사개요_층수지하" : 0,
    "공사개요_층수지상" : 0,
    "공사개요_굴착깊이" : 0.0,
    "공사개요_최고높이" : 0.0,
    "공사개요_연면적" : 0.0,
    "기타특수구조물개요" : "",
    "주요공법1" : "",
    "주요공법2" : "",
    "주요공법3" : "",
    "주요공법4" : "",
    "주요공법5" : "",
    "주요공법6" : "",
    "주요공법7" : "",
    "주요공법8" : "",
    "주요공법9" : "",
    "주요공법10" : "",
    "파일경로" : "",
    "비고" : "",
}

cursor_positions = []  # 문단 앞 커서 위치를 담는 리스트 (list, para, pos)
section_titles = []    # 각 위치에 대응하는 목차 또는 섹션 이름 **목차명=필드명이되도록할것
정렬된_cursor_positions=[]
정렬된_section_titles=[]
custom_data = []

def 안전관리계획서1_데이터_초기화():
    """
    공통 데이터를 초기화합니다.
    """
    for key in 안전계획서1_dic.keys():
        안전계획서1_dic[key] = "" if isinstance(안전계획서1_dic[key], str) else 0 if isinstance(안전계획서1_dic[key], int) else 0.0

def 커서_섹션명_커스텀_리스트_데이터_초기화():
    global cursor_positions
    global section_titles
    global custom_data 
    cursor_positions = []  # 문단 앞 커서 위치를 담는 리스트 (list, para, pos)
    section_titles = []    # 각 위치에 대응하는 목차 또는 섹션 이름 **목차명=필드명이되도록할것
    custom_data = []

#endregion

#################################################################

#region 데이터베이스 초기화 및 연결 함수
def connect_db():
    """
    SQLite 데이터베이스에 연결합니다.
    데이터베이스 파일 이름은 기본적으로 'data.db'로 고정됩니다.
    데이터베이스 파일이 없으면 자동으로 생성됩니다.

    :return: 데이터베이스 연결 객체와 커서
    """
    db_name = 'data.db'  # 고정된 데이터베이스 파일 이름
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()
    return conn, cursor

# 테이블 생성 함수
def initialize_db(table_name, data_dict):
    """
    딕셔너리 키를 기반으로 데이터베이스 테이블을 생성합니다.
    """
    conn = sqlite3.connect('data.db')
    cursor = conn.cursor()
    
    # 딕셔너리에서 열 생성 쿼리 동적 작성
    columns = ', '.join([f"{key} {get_sqlite_type(value)}" for key, value in data_dict.items()])
    fixed_columns = "섹션명 TEXT, 리스트 INTEGER, para INTEGER, pos INTEGER"
    query = f"CREATE TABLE IF NOT EXISTS {table_name} (id INTEGER PRIMARY KEY AUTOINCREMENT, {columns}, {fixed_columns})"
    
    cursor.execute(query)
    conn.commit()
    conn.close()

def get_sqlite_type(value):
    """
    딕셔너리 값의 데이터 타입에 따라 SQLite 데이터 타입 반환.
    """
    if isinstance(value, int):
        return "INTEGER"
    elif isinstance(value, float):
        return "REAL"
    elif isinstance(value, str):
        return "TEXT"
    else:
        return "TEXT"  # 기본값

# 데이터 삽입 함수 / 문서 전체를 한방에 db화함.
def insert_data(
    table_name, data_dict, 섹션명_리스트= 정렬된_section_titles, 데이터_리스트=정렬된_cursor_positions
):
    """
    딕셔너리와 리스트를 사용해 데이터베이스에 데이터를 삽입합니다.
    
    :param db_name: 데이터베이스 파일 이름
    :param table_name: 테이블 이름
    :param data_dict: 삽입할 공통 데이터 딕셔너리
    :param 섹션명_리스트: 섹션명을 담은 리스트
    :param 데이터_리스트: (리스트, para, pos) 튜플 리스트
    """
    if len(섹션명_리스트) != len(데이터_리스트):
        raise ValueError("섹션명_리스트와 데이터_리스트의 길이가 다릅니다.")
    
    conn = sqlite3.connect('data.db')
    cursor = conn.cursor()

    # 공통 데이터 열과 값
    common_columns = ', '.join(data_dict.keys())
    common_values = tuple(data_dict.values())

    # 섹션명_리스트와 데이터_리스트를 조합하여 삽입
    for 섹션명, (리스트, para, pos) in zip(섹션명_리스트, 데이터_리스트):
        query = f'''
            INSERT INTO {table_name} (
                {common_columns}, 섹션명, 리스트, para, pos
            ) VALUES ({', '.join(['?'] * (len(data_dict) + 4))})
        '''
        values = common_values + (섹션명, 리스트, para, pos)
        cursor.execute(query, values)

    conn.commit()
    conn.close()
    print(f"{table_name} 테이블에 데이터가 삽입되었습니다.")

# 모든 데이터 조회 함수
def print_table_data(table_name):
    conn = sqlite3.connect('data.db')
    cursor = conn.cursor() 

    cursor.execute(f"SELECT * FROM {table_name}")
    rows = cursor.fetchall()
    
    for row in rows:
        print(row)
    
    conn.close()

#custom_data로 특정id만 추출하기(id의 위치),(id+1의 위치)(id의 섹션명)
def fetch_custom_data_with_next_positions(table_name, id_list):
    """
    주어진 테이블 이름과 ID 리스트에 해당하는 데이터를 custom_data 형식으로 반환.
    (id의 시작 좌표), (id+1의 시작 좌표), "id 섹션명" 구조로 변환.

    :param table_name: 데이터베이스 테이블 이름
    :param id_list: ID 리스트
    :return: custom_data 리스트
    """
    conn = sqlite3.connect('data.db')
    cursor = conn.cursor()
    
    # ID 리스트를 조건으로 데이터 가져오기
    placeholders = ', '.join('?' for _ in id_list)
    query = f'''
        SELECT id, 리스트, para, pos, 섹션명 
        FROM {table_name}
        WHERE id IN ({placeholders}) 
        ORDER BY id
    '''
    cursor.execute(query, id_list)
    rows = cursor.fetchall()
 

    # ID+1 위치 한 번에 가져오기
    next_positions = {}
    for id_ in id_list:
        query_next = f'''
            SELECT 리스트, para, pos 
            FROM {table_name}
            WHERE id = ?
        '''
        cursor.execute(query_next, (id_ + 1,))  # `id + 1`으로 수정
        next_row = cursor.fetchone()
        if next_row:
            next_positions[id_] = next_row
        else:
            next_positions[id_] = "문서끝"

    # 데이터 변환
    custom_data = []
    for row in rows:
        current_id, 리스트, para, pos, 섹션명 = row
        next_position = next_positions.get(current_id, "문서끝")
        custom_data.append(
            ((리스트, para, pos), next_position, 섹션명)
        )
    
    conn.close()
    return custom_data

# 데이터 삭제 함수
def delete_data(table_name, row_id):
    """
    특정 테이블에서 특정 id 값을 가진 데이터를 삭제합니다.

    :param table_name: 테이블 이름
    :param row_id: 삭제할 행의 id 값
    """
    conn, cursor = connect_db() 
    query = f'DELETE FROM {table_name} WHERE id = ?'
    cursor.execute(query, (row_id,))
    conn.commit()
    conn.close()


# 데이터 업데이트 함수  ##이것도 잘모르겠음. 
def update_data(table_name, row_id, **kwargs):
    """
    특정 테이블에서 특정 ID의 데이터를 업데이트합니다.
    
    :param table_name: 테이블 이름
    :param row_id: 업데이트할 행의 ID
    :param kwargs: 업데이트할 열 이름과 값의 딕셔너리 (예: {"파일이름": "새 파일명", "현장명": "새 현장명"})
    """
    conn, cursor = connect_db()
    updates = []
    values = []

    for key, value in kwargs.items():
        updates.append(f"{key} = ?")
        values.append(value)
    
    values.append(row_id)
    query = f'UPDATE {table_name} SET {", ".join(updates)} WHERE id = ?'
    cursor.execute(query, values)
    conn.commit()
    conn.close()

#테이블 삭제
def reset_table(table_name):
    """
    특정 테이블을 삭제하고 초기화합니다.
    테이블 삭제만 처리하며, 생성은 별도의 함수로 처리해야 합니다.

    :param table_name: 초기화할 테이블 이름
    """
    conn, cursor = connect_db()
    
    # 테이블 삭제
    cursor.execute(f'DROP TABLE IF EXISTS {table_name}')
    conn.commit()
    conn.close()
    
    print(f"{table_name} 테이블이 초기화되었습니다.")

#열 목록 확인
def check_table_structure(table_name):
    conn = sqlite3.connect('data.db')
    cursor = conn.cursor()
    cursor.execute(f"PRAGMA table_info({table_name})")
    rows = cursor.fetchall()
    conn.close()
    
    print(f"{table_name} 테이블 구조:")
    for row in rows:
        print(row)
    
# 데이터베이스 초기화
initialize_db('con_1',안전계획서1_dic)

#endregion

#################################################################



In [ ]:
print_table_data('con_1')

In [ ]:
#region hwp 초기화

hwp = Hwp(new=True)
#hwp2 = Hwp(new=True)

hwp.Open(r"C:\Users\user\Desktop\02. 안전관리계획서(R2)-20250125T110710Z-001\02. 안전관리계획서(R2)\01. [변경] 안전관리계획서 제1편 - 천안 성성5지구 IPARK 신축공사(R4) = 정기안전점검 수정 240925.hwp")
#hwp2.Open(r"C:\Users\codek\Documents\GitHub\connwnw\test\샘플_템플릿.hwp")

hwp.MoveDocBegin()

def 오른_표이동(n):
    """n번 오른쪽으로 이동"""
    for _ in range(n):
        hwp.TableRightCell()
#캐럿 포지션 저장

#마지막 처음 추가하기
def 처음지점추가():
    hwp.MoveDocBegin()
    cursor_positions.append(hwp.GetPos())
    section_titles.append("문서처음")
def 문단시작지점추가():
        문단시작_list =[]
        for i in hwp.ctrl_list:
            if i.UserDesc == "새 번호":#컨트롤이 표일 경우
                문단시작_list.append(i)#리스트에 저장
        for i in range(len(문단시작_list)):
            hwp.move_to_ctrl(문단시작_list[i])
            hwp.MoveLeft()
            cursor_positions.append(hwp.GetPos())
            section_titles.append(f"문단{i+1}")
#마지막 지점 추가하기
def 마지막지점추가():
    hwp.MoveDocEnd()
    cursor_positions.append(hwp.GetPos())
    section_titles.append("문서끝")
#리스트 para기준 재정렬
def 재정렬_파라기준(cursor_positions, section_titles):
    """
    cursor_positions의 'para' 값을 기준으로 cursor_positions와 section_titles를 재정렬하는 함수.
    
    매개변수:
    - cursor_positions (list of tuples): (table, para, dotPos) 형식의 커서 위치 리스트.
    - section_titles (list of str): 해당 커서 위치에 대응하는 섹션 제목 리스트.
    
    반환값:
    - tuple: 정렬된 cursor_positions와 section_titles.
    """
    # cursor_positions와 section_titles를 결합
    결합_데이터 = list(zip(cursor_positions, section_titles))
    
    # 'para' 값(튜플의 두 번째 값)을 기준으로 정렬
    정렬된_데이터 = sorted(결합_데이터, key=lambda x: x[0][1])
    
    # 정렬된 데이터를 다시 두 개의 리스트로 분리
    정렬된_cursor_positions, 정렬된_section_titles = zip(*정렬된_데이터)
    
    return list(정렬된_cursor_positions), list(정렬된_section_titles)
def 고정폭빈칸삭제():
    hwp.HAction.GetDefault("DeleteCtrls", hwp.HParameterSet.HDeleteCtrls.HSet)
    hwp.HParameterSet.HDeleteCtrls.CreateItemArray("DeleteCtrlType", 1)
    hwp.HParameterSet.HDeleteCtrls.DeleteCtrlType.SetItem(0, 7)  # <--- Item을 SetItem으로 고쳤음.
    hwp.HAction.Execute("DeleteCtrls", hwp.HParameterSet.HDeleteCtrls.HSet)
#endregion

#################################################################

#region 문서 공통 info 추출
고정폭빈칸삭제()
#파일경로 추출
안전계획서1_dic["파일경로"] = hwp.Path

#파일이름 추출
안전계획서1_dic["파일이름"] = os.path.basename(안전계획서1_dic["파일경로"])
print(안전계획서1_dic["파일경로"], 안전계획서1_dic["파일이름"])

##처음 위치로##
hwp.Cancel()
hwp.MoveDocBegin()

# 정규표현식으로 연도와 월 추출
연월텍스트 = hwp.GetPageText()
패턴 = r'(\d{4})\.\s*(\d{2})'
매칭 = re.search(패턴, 연월텍스트)

if 매칭:
    안전계획서1_dic["보고서_날짜_년"], 안전계획서1_dic["보고서_날짜_월"] = 매칭.groups()
    #print(f"연도: {안전계획서1_dic["보고서_날짜_년"]}")
    #print(f"월: {안전계획서1_dic["보고서_날짜_월"]}")
else:
    print("연도와 월을 찾을 수 없습니다.")

#region###--공사개요서 뒤지기--###
if hwp.find_forward("1. 공사 개요서"):
    hwp.SelectCtrlFront()
    hwp.ShapeObjTableSelCell() # 첫번째 셀 선택
else : print("공사개요서 못찾음.")

#현장명
오른_표이동(2)
안전계획서1_dic["현장명"] = hwp.get_selected_text()

#소재지
오른_표이동(2)
안전계획서1_dic["현장소재지"] = hwp.get_selected_text()

#공사금액
오른_표이동(4)
안전계획서1_dic["공사금액"] = hwp.get_selected_text()

#시공자회사명
오른_표이동(3)
안전계획서1_dic["시공자"] = hwp.get_selected_text()

#설계자 회사명
오른_표이동(25)
안전계획서1_dic["설계자"] = hwp.get_selected_text()

#대상공사
오른_표이동(41)
안전계획서1_dic["공사개요_대상공사"] = hwp.get_selected_text()

#구조
오른_표이동(1)
안전계획서1_dic["공사개요_구조"] = hwp.get_selected_text()

#개소
오른_표이동(1)
안전계획서1_dic["공사개요_개소"] = hwp.get_selected_text()

오른_표이동(1)
안전계획서1_dic["공사개요_층수지하"] = hwp.get_selected_text()

오른_표이동(1)
안전계획서1_dic["공사개요_층수지상"] = hwp.get_selected_text()

오른_표이동(1)
안전계획서1_dic["공사개요_굴착깊이"] = hwp.get_selected_text()

오른_표이동(1)
안전계획서1_dic["공사개요_최고높이"] = hwp.get_selected_text()

오른_표이동(1)
안전계획서1_dic["공사개요_연면적"] = hwp.get_selected_text()

오른_표이동(2)
안전계획서1_dic["기타특수구조물개요"] = hwp.get_selected_text()

#공법리스트
오른_표이동(2)
주요공법 = hwp.get_selected_text()  # 텍스트 가져오기
공법리스트 = [공법.strip() for 공법 in 주요공법.split('\n') if 공법.strip()]  # 줄 단위로 나누고 양쪽 공백 제거 및 빈 줄 제거
print(공법리스트)
for i in range(1, 11):  # "주요공법1" ~ "주요공법10"
        if i <= len(공법리스트):  # 공법리스트에 값이 남아 있는 경우
            안전계획서1_dic[f"주요공법{i}"] = 공법리스트[i - 1]
        else:  # 공법리스트에 값이 없는 경우 빈 문자열 유지
            안전계획서1_dic[f"주요공법{i}"] = ""
print(안전계획서1_dic)
#endregion

안전계획서1_dic["비고"] = ""  # 비고 초기화

hwp.Cancel()

#endregion

#################################################################

#region 섹션명추출
def 섹션명_추출(hwp, save_temp_path="템플릿\\Temp.hwp"):
    """
    HWP 문서에서 섹션 타이틀 추출 및 정리 작업 수행.
    
    :param hwp: 한글(HWP) 객체
    :param save_temp_path: 임시 저장 경로 (기본값: "템플릿\\Temp.hwp")
    """
    # 1. 템플릿 파일 저장
    hwp.SaveAs(save_temp_path, arg="lock:false")
    hwp3 = Hwp(new= True, visible=True)
    hwp3.open(save_temp_path)
    # 2. 모든 표 삭제
    for ctrl in reversed(hwp3.ctrl_list):
        if ctrl.UserDesc == "표":  # 컨트롤이 표일 경우
            hwp3.delete_ctrl(ctrl)  # 바로 삭제
    #고정폭빈칸삭제
    hwp3.HAction.GetDefault("DeleteCtrls", hwp3.HParameterSet.HDeleteCtrls.HSet)
    hwp3.HParameterSet.HDeleteCtrls.CreateItemArray("DeleteCtrlType", 1)
    hwp3.HParameterSet.HDeleteCtrls.DeleteCtrlType.SetItem(0, 7)  # <--- Item을 SetItem으로 고쳤음.
    hwp3.HAction.Execute("DeleteCtrls", hwp3.HParameterSet.HDeleteCtrls.HSet)
    # 3. 문서의 공백 및 불필요한 내용 정리
    hwp3.MoveDocBegin()
    while hwp3.MoveSelRight():
        selected_text = hwp3.get_selected_text()
        if not selected_text.strip():  # 공백 문자열 삭제
            hwp3.Delete()
        elif hwp3.MoveNextParaBegin():
            continue
        else:
            break
    
    # 4. 강제쪽나눔 전체 삭제##todo 코드 변경 실패함.
    def delete_forced_page_breaks():
        pset = hwp3.HParameterSet.HGotoE
        hwp3.HAction.GetDefault("Goto", pset.HSet)
        while True:
            try:
                pset.HSet.SetItem("DialogResult", 54)  # 강제쪽나눔으로 이동
                pset.SetSelectionIndex = 5
                if not hwp3.HAction.Execute("Goto", pset.HSet):  # 이동 실패 시 종료
                    break
                hwp3.DeleteBack()  # 강제쪽나눔 삭제
            except Exception as e:
                print(f"오류 발생: {e}")
                break
    hwp3.MoveDocBegin()
    delete_forced_page_breaks()
    

    # 5. 텍스트 스캔 및 섹션 타이틀 추출
    hwp3.MoveDocBegin()
    hwp3.init_scan()
    extracted_texts = []  # 추출된 텍스트 저장
    while True:
        state, text = hwp3.get_text()
        if text and text.strip():  # 공백 제외
            clean_text = text.replace("\r\n", "").replace("\n", "").replace("\r", "")
            print(clean_text)  # 정리된 텍스트 출력
            extracted_texts.append(clean_text)
        if state <= 1:  # 종료 조건
            break
    hwp3.release_scan()

    # 6. 결과 반환 (추출된 텍스트 리스트)
    return extracted_texts

# hwp 객체를 가져와서 함수 호출
추출한_섹션명_리스트 = 섹션명_추출(hwp)
print(추출한_섹션명_리스트)
# 추출된 텍스트 확인
for title in 추출한_섹션명_리스트:
    print(title)

#endregion

#################################################################

#region 문서내 포지션 추출
#처음위치추가
처음지점추가()

마지막지점추가()

문단시작지점추가()


#문서의 내용 리스트에 추가
#findforward로 섹션리스트를 순회하면서 위치 추가
def 위치추가():
    # 목차 문단 맨 앞의 위치 가져오기
    pos = hwp.GetPos()  # pos는 (리스트, para, pos) 형태의 튜플
    
    # 리스트 값이 0인지 확인
    if pos[0] != 0:  # 리스트 값이 0이 아니면 함수 종료
        print(f"리스트 값이 0이 아니므로 추가하지 않음: {pos}")
        #다시 찾기 추가해야함.
        return
    cursor_positions.append(hwp.GetPos())

    #목차 문단 셀선택 후 추가하고 다시 첫 위치로 돌아가기
    hwp.MoveSelParaEnd()
    section_titles.append(hwp.get_selected_text())
    hwp.MoveParaBegin()
def 중간위치추가(sec_list):
    hwp.MoveDocBegin()
    for section in sec_list:
        while hwp.find(section, 'AllDoc', WholeWordOnly=1, SeveralWords=0, UseWildCards=0):
            # 현재 위치의 커서 정보를 가져옴
            
            hwp.MoveLeft()  # 커서를 찾은 위치로 보정
            hwp.MoveRight()
            pos = hwp.GetPos()

            if pos[0] != 0:  # 리스트 값이 0이 아니면 다음 위치로 검색
                print(f"리스트 값이 0이 아니므로 다음 위치 검색: {pos}")
                hwp.MoveRight()  # 커서를 한 칸 오른쪽으로 이동하여 다음 검색 준비
                continue  # 다음 위치 검색
            else:
                # 커서 위치가 유효하다면, 데이터를 추가
                cursor_positions.append(pos)
                hwp.MoveSelParaEnd()  # 현재 커서가 위치한 문단의 끝으로 이동
                section_titles.append(hwp.get_selected_text())
                print(f'{section} : 완료')
                break  # 섹션 검색 완료 후 다음 섹션으로 이동
        else:
            print(f"{section} : 실패")  # 검색 실패 처리
중간위치추가(추출한_섹션명_리스트)
# print(cursor_positions, section_titles)

# #para 크기 순으로 재정렬
정렬된_cursor_positions, 정렬된_section_titles = 재정렬_파라기준(cursor_positions, section_titles)
# print(정렬된_cursor_positions, 정렬된_section_titles)

#endregion

#################################################################
 
#region 복붙하기 

#내용 삽입 함수
def 복붙_내용삽입(custom_data):
    """
    custom_data: [(섹션시작좌표, 섹션끝좌표, 섹션명), ...] 형태의 리스트
    """
    for section_start, section_end, section_name in custom_data:
        # 섹션 시작 위치로 이동
        hwp.SetPos(*section_start)  
        hwp.MoveNextParaBegin()  # 다음 단락으로 이동
        hwp.Select()  # 섹션 시작 위치에서 선택 시작

        # 섹션 끝 위치로 이동
        if section_end == None:
            print("잘못된 id를 추출하였습니다.(문서끝id입력함)")
        else:
            hwp.SetPos(*section_end)
            hwp.MoveLeft()  # 끝 위치에서 한 글자 왼쪽으로 이동해 선택 범위 조정

        # 복사 작업 수행
        hwp.Copy()

        # 두 번째 HWP 파일의 필드로 이동하여 붙여넣기
        #hwp2.MoveToField(section_name)
        #hwp2.Paste()  # 복사한 내용 붙여넣기

#복붙_내용삽입(custom_data)
#endregion

#################################################################

#region 필드추가 / 템플릿 제작하기
def 필드만들기(list): #todo : 미완성
    """
    섹션명만 남긴 한글 파일에서 한줄 엔터 후 '섹션명'을 필드로 만듦(템플릿제작용)
    list : 추출한 섹션명 리스트를 넣음
    """
    for i in list:
        if hwp.find_forward(i):
            hwp.MoveParaEnd()
            hwp.BreakPara()
            hwp.create_field(name=i, direction=i)
#endregion

In [11]:
print(정렬된_section_titles)
print(len(정렬된_section_titles), len(정렬된_cursor_positions))
print(정렬된_cursor_positions)
print(안전계획서1_dic)

['문서처음', '문서처음', '2024. 09', '목     차 ', '목     차 ', '문단1', '문단1', '1. 공사 개요서', '2. 현장 위치도', '2-1. 현장 위치도', '2-2. 현장 주변현황', '1) 현장 주변현황', '3. 전체 공정표', '■ 공사 예정공정표 - 후면 첨부참조', '4. 공사 설계도면', '■ 건축/구조/토목 설계도서 - 별도 첨부참조', '5. 건설물ㆍ공사용 기계설비등의 배치를 나타내는 도면서류', '5-1. 가설구조물 등의 설치 및 해체계획', '■ 공사용 가설물 배치 계획도 - 후면 첨부참조', '5-2. 공종별 장비, 기계 투입계획', '1) 공통공사', '2) 굴착 및 흙막이지보공', '3) 건축(구조물) 공사', '4) 건축(마감) 공사', '문단2', '문단2', '1. 현장 여건 분석', '1-1. 지형 및 지질 분석 [소규모 지하안전영향평가서 발췌]', '1) 지형분석', '2) 지질분석', '3) 구지형 분석', '(1) 구지형도 분석', '(2) 위성사진에 의한 지형분석', '1-2. 시추조사 [소규모 지하안전영향평가서 발췌]', '1) 시추조사 결과', '(1) 지층분포현황', '(2) 시추조사에 의한 지층분석 결과', '(3) 지층분포 현황', '(4) 지층단면도 및 분포특성', '(5) 지층별 분포특성 분석결과', '2) 현장시험결과 분석', '(1) 표준관입시험 결과', '(2) 공내 지하수위 측정결과', '① 지하수위 측정 결과', '② 설계 지하수위', '③ 현장 투수시험 결과', '④ 현장 암반수압시험 결과', '⑤ 공내전단시험 결과', '⑥ 공내재하시험 결과(P.M.T)', '3) 설계지하수위', '(1) 강우에 따른 지하수위 상승고 산정', '① 연도별 일 최대 강수량(천안기상관측소, 단위 : mm)', '② 일 최대 강우량 발생 후 30일간 강우량(1995년, 천안기상관측소, 단위 : mm)', '③ 지표면 조건에 따른 강우 침투율', '▌검토단면 A-A', '▌A-A 단면 

In [19]:
insert_data('con_1',안전계획서1_dic,정렬된_section_titles,정렬된_cursor_positions)

con_1 테이블에 데이터가 삽입되었습니다.


In [ ]:
check_table_structure('con_1')

In [10]:
cursor_positions.clear()
정렬된_cursor_positions.clear()
정렬된_section_titles.clear()
section_titles.clear()

In [ ]:
print_table_data('con_1')

In [22]:
#region db에서 custom_data 특정 id만 추출
selected_ids = [55,60,70]  # 사용자가 선택한 ID
custom_data = fetch_custom_data_with_next_positions('con_1',selected_ids)
print(custom_data)
#endregion

[((0, 229, 3), (0, 232, 3), '▌검토단면 A-A'), ((0, 246, 3), (0, 250, 3), '▌C-C 단면 침투해석 결과'), ((0, 276, 1), (0, 277, 2), '1-3. 현장 주변현황 분석 [소규모 지하안전영향평가서 발췌]')]


In [ ]:
%gui qt

import sys
import sqlite3
import pandas as pd
from PyQt5.QtWidgets import (
    QApplication, QMainWindow, QVBoxLayout, QLineEdit, QTableWidget, QTableWidgetItem,
    QComboBox, QWidget, QLabel, QPushButton, QHBoxLayout, QFileDialog
)
from PyQt5.QtCore import Qt

# QApplication 중복 생성 방지
app = QApplication.instance()  # 현재 QApplication 인스턴스 확인
if not app:  # 없다면 새로 생성
    app = QApplication(sys.argv)

class SQLiteApp(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("SQLite DB 테이블 보기")
        self.setGeometry(100, 100, 800, 600)

        # 데이터베이스 연결
        self.conn = sqlite3.connect("data.db")
        self.cursor = self.conn.cursor()

        # UI 레이아웃 설정
        self.layout = QVBoxLayout()

        # 드롭다운(테이블 선택)
        self.table_selector_label = QLabel("테이블을 선택하세요:")
        self.layout.addWidget(self.table_selector_label)

        self.table_selector = QComboBox()  # QTableWidget → QComboBox로 수정
        self.layout.addWidget(self.table_selector)
        self.table_selector.currentIndexChanged.connect(self.load_table_data)  # 선택 변경 시 함수 연결

        # 사용자 입력 필드
        input_layout = QHBoxLayout()
        self.input_label = QLabel("입력할 ID 입력 (쉼표로 구분):")
        input_layout.addWidget(self.input_label)

        self.id_input = QLineEdit()
        self.id_input.setPlaceholderText("예: 55, 60, 70")
        input_layout.addWidget(self.id_input)

        # Fetch 및 복붙 버튼
        self.fetch_button = QPushButton("데이터 가져오기")
        self.fetch_button.clicked.connect(self.fetch_custom_data)
        input_layout.addWidget(self.fetch_button)

        self.paste_button = QPushButton("복붙 내용 삽입")
        self.paste_button.clicked.connect(self.execute_paste)
        self.paste_button.setEnabled(False)  # 초기 상태에서는 비활성화
        input_layout.addWidget(self.paste_button)

        self.layout.addLayout(input_layout)

        # 내부 상태
        self.selected_ids = []  # 사용자가 입력한 ID
        self.custom_data = []  # fetch_custom_data_with_next_positions 결과 저장

        # 테이블 위젯 생성
        self.table_widget = QTableWidget()
        self.layout.addWidget(self.table_widget)
        
        #버튼 레이아웃
        self.button_layout = QHBoxLayout()
        
        # 열 숨기기 버튼
        self.hide_button = QPushButton("선택한 열 숨기기")
        self.hide_button.clicked.connect(self.hide_selected_column)
        self.button_layout.addWidget(self.hide_button)

        # 열 보이기 버튼
        self.show_button = QPushButton("모든 열 보이기")
        self.show_button.clicked.connect(self.show_all_columns)
        self.button_layout.addWidget(self.show_button)
        
        self.layout.addLayout(self.button_layout)

        #버튼2 레이아웃
        self.button2_layout = QHBoxLayout()

        # 엑셀 내보내기 버튼
        self.export_button = QPushButton("엑셀로 내보내기")
        self.export_button.clicked.connect(self.export_to_excel)
        self.button2_layout.addWidget(self.export_button)

        # 엑셀 불러오기 버튼
        self.import_button = QPushButton("엑셀에서 불러오기")
        self.import_button.clicked.connect(self.import_from_excel)
        self.button2_layout.addWidget(self.import_button)

        self.layout.addLayout(self.button2_layout)
        

        # 메인 위젯 설정
        container = QWidget()
        container.setLayout(self.layout)
        self.setCentralWidget(container)

        # 초기 테이블 목록 로드
        self.load_table_list()

    def load_table_list(self):
        """DB의 테이블 목록을 드롭다운에 로드"""
        self.cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = [row[0] for row in self.cursor.fetchall()]
        self.table_selector.addItems(tables)

    def load_table_data(self):
        """선택한 테이블의 데이터를 로드하여 QTableWidget에 표시"""
        selected_table = self.table_selector.currentText()
        if not selected_table:
            return

        # 선택된 테이블 데이터 가져오기
        self.cursor.execute(f"SELECT * FROM {selected_table}")
        rows = self.cursor.fetchall()
        columns = [description[0] for description in self.cursor.description]

        # QTableWidget 설정
        self.table_widget.setRowCount(len(rows))
        self.table_widget.setColumnCount(len(columns))
        self.table_widget.setHorizontalHeaderLabels(columns)
        self.table_widget.horizontalHeader().setSectionsMovable(True)  # 열 이동 가능 설정

        # 데이터 삽입
        for i, row in enumerate(rows):
            for j, value in enumerate(row):
                self.table_widget.setItem(i, j, QTableWidgetItem(str(value)))

        #  # 초기 상태에서 숨길 열 설정
        # visible_columns = ["column1", "column2"]  # 표시할 열 이름
        # for col_index, col_name in enumerate(columns):
        #     if col_name not in visible_columns:
        #         self.table_widget.setColumnHidden(col_index, True)  # 숨기기

        # 초기 상태에서 숨길 열 설정
        hidden_columns = ["파일이름", "공사개요_대상공사", "공사개요_구조", "공사개요_개소", "공사개요_층수지상", "공사개요_층수지하", "파일경로", "리스트", "para", "pos"]  # 숨길 열 이름
        for col_index, col_name in enumerate(columns):
            if col_name in hidden_columns:
                self.table_widget.setColumnHidden(col_index, True)  # 숨기기

    def hide_selected_column(self):
        """선택한 열 숨기기"""
        # 현재 선택된 열을 가져옴
        selected_columns = self.table_widget.selectedIndexes()
        if selected_columns:
            column_indices = {index.column() for index in selected_columns}  # 중복 제거
            for column_index in column_indices:
                self.table_widget.setColumnHidden(column_index, True)

    def show_all_columns(self):
        """모든 숨겨진 열 보이기"""
        for column_index in range(self.table_widget.columnCount()):
            self.table_widget.setColumnHidden(column_index, False)

    def export_to_excel(self):
        """현재 선택된 테이블 데이터를 엑셀로 내보내기"""
        selected_table = self.table_selector.currentText()
        if not selected_table:
            return

        # 테이블 데이터 가져오기
        self.cursor.execute(f"SELECT * FROM {selected_table}")
        rows = self.cursor.fetchall()
        columns = [description[0] for description in self.cursor.description]

        # Pandas DataFrame 생성
        df = pd.DataFrame(rows, columns=columns)

        # 파일 저장 다이얼로그
        file_path, _ = QFileDialog.getSaveFileName(self, "엑셀로 저장", "", "Excel Files (*.xlsx);;All Files (*)")
        if file_path:
            df.to_excel(file_path, index=False)
            print(f"엑셀 파일로 저장됨: {file_path}")

    def import_from_excel(self):
        """엑셀 파일에서 데이터를 읽어와 DB에 반영"""
        selected_table = self.table_selector.currentText()
        if not selected_table:
            return

        # 파일 열기 다이얼로그
        file_path, _ = QFileDialog.getOpenFileName(self, "엑셀 파일 열기", "", "Excel Files (*.xlsx);;All Files (*)")
        if not file_path:
            return

        # 엑셀 파일 읽기
        df = pd.read_excel(file_path)

        # DB에 반영 - 기존 데이터 삭제 후 새로 삽입
        self.cursor.execute(f"DELETE FROM {selected_table}")
        for _, row in df.iterrows():
            placeholders = ", ".join(["?"] * len(row))
            self.cursor.execute(f"INSERT INTO {selected_table} VALUES ({placeholders})", tuple(row))

        self.conn.commit()
        print(f"엑셀 파일의 내용이 {selected_table} 테이블에 반영됨.")
        self.load_table_data()  # 테이블 데이터 갱신

    def fetch_custom_data(self):
        """사용자 입력을 처리하고 외부 함수 호출"""
        # 사용자 입력 값 처리
        input_text = self.id_input.text()
        try:
            self.selected_ids = [int(x.strip()) for x in input_text.split(",") if x.strip().isdigit()]
            if not self.selected_ids:
                raise ValueError("숫자를 입력해야 합니다.")
        except ValueError as e:
            print(f"입력 오류: {e}")
            return

        print(f"선택된 ID: {self.selected_ids}")

        # 외부 함수 호출하여 데이터 가져오기
        table_name = "con_1"  # 예제 테이블 이름
        self.custom_data = fetch_custom_data_with_next_positions(table_name, self.selected_ids)
        print(f"가져온 데이터: {self.custom_data}")

        # 버튼 활성화
        if self.custom_data:
            self.paste_button.setEnabled(True)

    def execute_paste(self):
        """복붙 내용 삽입 버튼 클릭 시 외부 함수 호출"""
        if not self.custom_data:
            print("가져온 데이터가 없습니다.")
            return

        print("복붙 내용 삽입 수행 중...")
        복붙_내용삽입(self.custom_data)

    def closeEvent(self, event):
        """창 닫기 이벤트 처리 - DB 연결 닫기"""
        self.conn.close()
        event.accept()


if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = SQLiteApp()
    window.show()
    sys.exit(app.exec_())


In [7]:
import sys
import sqlite3
import pandas as pd
from PyQt5.QtWidgets import (
    QApplication, QVBoxLayout, QLineEdit, QTableWidget, QTableWidgetItem,
    QComboBox, QWidget, QLabel, QPushButton, QHBoxLayout, QFileDialog, QMainWindow, QMessageBox
)
from PyQt5.QtCore import Qt

# 전역 변수 선언
app = None  # PyQt5 애플리케이션 객체
conn = None  # SQLite 데이터베이스 연결 객체
cursor = None  # SQLite 커서 객체
table_selector = None  # 테이블 선택 드롭다운
table_widget = None  # 테이블 데이터를 표시하는 위젯
id_input = None  # 사용자 입력 필드
fetch_button = None  # 데이터 가져오기 버튼
paste_button = None  # 복붙 내용 삽입 버튼
selected_ids = []  # 사용자가 입력한 ID 목록
custom_data = []  # 외부 함수에서 가져온 데이터 저장

# 메시지 박스
def show_message(title, message, info=None, icon=QMessageBox.Information):
    """
    PyQt5 메시지 박스를 표시하는 함수.
    :param title: 메시지 박스 제목
    :param message: 메시지 내용
    :param info: 추가 정보 (선택 사항)
    :param icon: 메시지 박스 아이콘 (기본값: Information)
    """
    msg = QMessageBox()
    msg.setIcon(icon)  # 메시지 아이콘 설정
    msg.setWindowTitle(title)  # 창 제목
    msg.setText(message)  # 메시지 내용 설정
    if info:
        msg.setInformativeText(info)  # 추가 정보 (선택 사항)
    msg.exec_()  # 메시지 박스 실행

# DB 연결 초기화
def init_db(db_path="data.db"):
    """
    SQLite 데이터베이스에 연결합니다.
    :param db_path: 데이터베이스 파일 경로
    """
    global conn, cursor
    conn = sqlite3.connect(db_path)  # 데이터베이스 연결 생성
    cursor = conn.cursor()  # 커서 객체 생성

# 테이블 목록 로드
def load_table_list():
    """
    SQLite 데이터베이스에서 테이블 목록을 가져와 드롭다운에 추가합니다.
    """
    global cursor, table_selector
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")  # 테이블 목록 조회
    tables = [row[0] for row in cursor.fetchall()]  # 결과에서 테이블 이름 추출
    table_selector.addItems(tables)  # 드롭다운에 테이블 목록 추가

# 선택한 테이블 데이터 로드
def load_table_data():
    """
    드롭다운에서 선택한 테이블의 데이터를 QTableWidget에 로드합니다.
    """
    global cursor, table_selector, table_widget
    selected_table = table_selector.currentText()  # 선택된 테이블 이름 가져오기
    if not selected_table:
        return

    # 테이블 데이터 조회
    cursor.execute(f"SELECT * FROM {selected_table}")
    rows = cursor.fetchall()  # 테이블의 모든 행 가져오기
    columns = [description[0] for description in cursor.description]  # 컬럼 이름 가져오기

    # QTableWidget에 데이터 삽입
    table_widget.setRowCount(len(rows))
    table_widget.setColumnCount(len(columns))
    table_widget.setHorizontalHeaderLabels(columns)  # 열 제목 설정

    for i, row in enumerate(rows):
        for j, value in enumerate(row):
            table_widget.setItem(i, j, QTableWidgetItem(str(value)))  # 각 셀에 데이터 삽입

    # 초기 숨김 설정 - 특정 열 숨기기
    hidden_columns = ["파일이름", "파일경로", "리스트", "para", "pos"]
    for col_index, col_name in enumerate(columns):
        if col_name in hidden_columns:
            table_widget.setColumnHidden(col_index, True)

    # show_message(
    #     title="테이블 로드 완료",
    #     message=f"'{selected_table}' 테이블의 데이터를 로드했습니다.",
    #     info=f"총 {len(rows)}개의 행이 로드되었습니다."
    # )

# Fetch 버튼 동작
def fetch_custom_data():
    """
    사용자가 입력한 ID를 처리하고 외부 함수로 데이터를 가져옵니다.
    """
    global id_input, selected_ids, custom_data, paste_button

    # 사용자 입력 처리
    input_text = id_input.text()
    try:
        # 쉼표로 구분된 숫자 리스트 생성
        selected_ids = [int(x.strip()) for x in input_text.split(",") if x.strip().isdigit()]
        if not selected_ids:
            raise ValueError("숫자를 입력해야 합니다.")
    except ValueError as e:
        show_message(
            title="입력 오류",
            message="유효한 숫자를 입력해야 합니다.",
            info=str(e),
            icon=QMessageBox.Warning
        )
        return

    show_message(
        title="ID 선택 완료",
        message="선택된 ID가 처리되었습니다.",
        info=f"선택된 ID: {selected_ids}"
    )

    # 외부 함수 호출 - 예제
    table_name = "con_1"
    custom_data = fetch_custom_data_with_next_positions(table_name, selected_ids)  # 외부 함수 호출
    if custom_data:
        # custom_data에서 섹션명(튜플의 마지막 요소)만 추출
        section_names = [item[-1] for item in custom_data]  # 각 항목의 마지막 요소 추출
        section_names_str = "\n".join(section_names)  # 줄바꿈으로 섹션명을 연결

        show_message(
            title="데이터 가져오기 완료",
            message="데이터를 성공적으로 가져왔습니다.",
            info=f"가져온 섹션명:\n{section_names_str}"
        )
        paste_button.setEnabled(True)
    else:
        show_message(
            title="데이터 없음",
            message="선택한 ID에 대한 데이터를 찾을 수 없습니다.",
            icon=QMessageBox.Warning
        )

# Paste 버튼 동작
def execute_paste():
    """
    외부 함수를 호출하여 복붙 데이터를 삽입합니다.
    """
    global custom_data
    if not custom_data:
        show_message(
            title="삽입 실패",
            message="삽입할 데이터가 없습니다.",
            icon=QMessageBox.Warning
        )
        return

    show_message(
        title="삽입 성공",
        message="데이터를 성공적으로 삽입했습니다.",
        info=f"삽입된 데이터: {custom_data}"
    )
    복붙_내용삽입(custom_data)  # 외부 함수 호출

# 엑셀 내보내기
def export_to_excel():
    """
    현재 선택된 테이블 데이터를 엑셀 파일로 내보냅니다.
    """
    global table_selector, cursor
    selected_table = table_selector.currentText()  # 선택된 테이블 이름 가져오기
    if not selected_table:
        return

    # 데이터 가져오기
    cursor.execute(f"SELECT * FROM {selected_table}")
    rows = cursor.fetchall()
    columns = [description[0] for description in cursor.description]

    # Pandas DataFrame으로 변환
    df = pd.DataFrame(rows, columns=columns)

    # 파일 저장 다이얼로그 열기
    file_path, _ = QFileDialog.getSaveFileName(None, "엑셀로 저장", "", "Excel Files (*.xlsx);;All Files (*)")
    if file_path:
        df.to_excel(file_path, index=False)  # 엑셀로 저장
        show_message(
            title="엑셀 내보내기 완료",
            message="데이터를 엑셀 파일로 내보냈습니다.",
            info=f"파일 경로: {file_path}"
        )

# 엑셀 불러오기
def import_from_excel():
    """
    엑셀 파일에서 데이터를 읽어와 선택된 테이블에 삽입합니다.
    """
    global cursor, conn, table_selector
    selected_table = table_selector.currentText()  # 선택된 테이블 이름 가져오기
    if not selected_table:
        return

    # 파일 열기 다이얼로그
    file_path, _ = QFileDialog.getOpenFileName(None, "엑셀 파일 열기", "", "Excel Files (*.xlsx);;All Files (*)")
    if not file_path:
        return

    # 엑셀 파일 읽기
    df = pd.read_excel(file_path)

    # 기존 데이터 삭제 및 새 데이터 삽입
    cursor.execute(f"DELETE FROM {selected_table}")
    for _, row in df.iterrows():
        placeholders = ", ".join(["?"] * len(row))
        cursor.execute(f"INSERT INTO {selected_table} VALUES ({placeholders})", tuple(row))

    conn.commit()
    show_message(
        title="엑셀 불러오기 완료",
        message="엑셀 데이터를 성공적으로 DB에 반영했습니다.",
        info=f"파일 경로: {file_path}"
    )
    load_table_data()  # 테이블 데이터 갱신

# UI 생성
def create_ui():
    """
    PyQt5 기반의 UI를 생성하고 초기 설정을 수행합니다.
    """
    global app, table_selector, table_widget, id_input, fetch_button, paste_button

    app = QApplication(sys.argv)  # QApplication 생성

    window = QMainWindow()
    window.setWindowTitle("SQLite DB 테이블 보기")  # 창 제목 설정
    window.setGeometry(100, 100, 800, 600)  # 창 크기 설정

    layout = QVBoxLayout()  # 전체 레이아웃

    # 테이블 선택 드롭다운
    table_selector_label = QLabel("테이블을 선택하세요:")
    layout.addWidget(table_selector_label)

    table_selector = QComboBox()
    layout.addWidget(table_selector)
    table_selector.currentIndexChanged.connect(load_table_data)

    # 사용자 입력 필드
    input_layout = QHBoxLayout()
    input_label = QLabel("입력할 ID 입력 (쉼표로 구분):")
    input_layout.addWidget(input_label)

    id_input = QLineEdit()
    id_input.setPlaceholderText("예: 55, 60, 70")  # 힌트 텍스트
    input_layout.addWidget(id_input)

    fetch_button = QPushButton("데이터 가져오기")
    fetch_button.clicked.connect(fetch_custom_data)  # 버튼 클릭 시 fetch_custom_data 호출
    input_layout.addWidget(fetch_button)

    paste_button = QPushButton("복붙 내용 삽입")
    paste_button.clicked.connect(execute_paste)  # 버튼 클릭 시 execute_paste 호출
    paste_button.setEnabled(False)  # 초기 상태 비활성화
    input_layout.addWidget(paste_button)

    layout.addLayout(input_layout)

    # 테이블 위젯
    table_widget = QTableWidget()
    layout.addWidget(table_widget)

    # 엑셀 관련 버튼
    button_layout = QHBoxLayout()
    export_button = QPushButton("엑셀로 내보내기")
    export_button.clicked.connect(export_to_excel)  # 엑셀 내보내기 연결
    button_layout.addWidget(export_button)

    import_button = QPushButton("엑셀에서 불러오기")
    import_button.clicked.connect(import_from_excel)  # 엑셀 불러오기 연결
    button_layout.addWidget(import_button)

    layout.addLayout(button_layout)

    # 메인 위젯 설정
    container = QWidget()
    container.setLayout(layout)
    window.setCentralWidget(container)

    return window

if __name__ == "__main__":
    init_db()  # 데이터베이스 초기화
    main_window = create_ui()  # UI 생성
    load_table_list()  # 테이블 목록 로드
    main_window.show()  # 메인 창 표시
    sys.exit(app.exec_())  # 이벤트 루프 실행


SystemExit: 0